## FIFA spelers score
### 1. 

**Importeer en bekijke de datasets 'players_19.csv' en 'players_20.csv'**

In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)

In [4]:
# !pip install pipreqs



In [ ]:
import numpy as np
import pandas as pd

dp_19=pd.read_csv('players_19.csv')
dp_20=pd.read_csv('players_20.csv')

dp_19.head()
dp_20.head()


### 2. 

Reminder video: https://datalab01.ict.hva.nl/w/115h1kbnT5B4eijPd6Gc95

In [ ]:
dp_19.groupby("club")["overall"].mean().sort_values(ascending=False).head(10)

en de uitkomst is :
Juventus             82.160000
FC Barcelona         80.285714
Argentina            80.000000
Napoli               79.840000
Inter                79.708333
FC Bayern München    78.500000
Milan                78.034483
Real Madrid          77.848485
Sweden               77.666667
Manchester United    77.454545
Name: overall, dtype: float64

### 3. 
**Selecteer van beide datasets alleen de kolommen 'sofifa_id', 'club' en 'overall'. Geef de kolommen
'overall' en 'club' in de dataset players_20 een nieuwe naam.  **

In [ ]:
# Rename columns, tip rename()
dp_20.rename(columns={'overall': 'totaal', 'club': 'groepje'}, inplace=True)
print(dp_20.columns)


**Voeg de datasets players_19 en
players_20 samen middels een left_join.** <br>
Video samen voegen datasets: https://datalab01.ict.hva.nl/w/fD7W7PsfkpiKovcTidMCac

In [ ]:
merged_df = pd.merge(dp_19, dp_20, left_on='sofifa_id', right_on='sofifa_id', how='left')
print(merged_df)

**Doe hetzelfde met een inner_join, wat is het
verschil tussen de datasets?**

In [ ]:
inner_df = pd.merge(
    dp_19,
    dp_20,
    on='sofifa_id',   
    how='inner',     
    suffixes=('_19','_20')  
)

print(inner_df.head())  

**Hoeveel unieke voetballers zitten er in beide datasets?
Hoeveel voetballers komen in beide datasets voor?**

In [ ]:
aantal_unieke_spelers = len(dp_20['sofifa_id'].unique())
print(aantal_unieke_spelers)

en 

aantal_unieke_spelers = len(dp_19['sofifa_id'].unique())
print(aantal_unieke_spelers)

### 4. 
**Gebruik de uitkomt van de left join en bepaal per speler de maximale overall ranking (dat is dus de overall ranking uit 2019 of uit 2020) en sla deze op in een nieuwe variabele best_overall**

*tip: gebruik numpy's maximum() function of pandas.DataFrame.max met axis = 1*

In [ ]:
merged_df['hoogste_overall'] = np.maximum(merged_df['overall_19'], merged_df['overall_20'])
top_speler = merged_df[merged_df['hoogste_overall'] == merged_df['hoogste_overall'].max()]
print(top_speler[[

### 5. 
**Welke spelers spelen niet voor dezelfde club in 2020? Tag deze spelers in een nieuwe
variabele “Transfer”. Hebben deze spelers gemiddeld gezien een hogere ranking in 2020 dan
in 2019?**

We gebruiken het resultaat van de inner join want dan hebben we direct alle spelers die zowel in 2019 als in 2020 een club hadden.

In [ ]:
inner_df = dp_19.merge(dp_20, how='inner', on='sofifa_id')
inner_df['transfer'] = inner_df['club_x'] != inner_df['club_y']
inner_df['transfer'] = inner_df['transfer'].map({True: 'Ja', False: 'Nee'})
print(inner_df[['short_name_x', 'club_x', 'club_y', 'transfer']])


### 6. 
**Selecteer middels een join de spelers die niet in beide datasets voorkomen. Welke join
moeten we hiervoor gebruiken?**

In [ ]:
# 
merged_df = dp_19.merge(dp_20, on='sofifa_id', how='outer', indicator=True)


niet_in_beide = merged_df[merged_df['_merge'] != 'both']


print(niet_in_beide[['sofifa_id', '_merge']])


## Spotify top 200 data - gebruik van window functies
### 7. 
**Laad de dataset SpotifyNL in. Zet de dataset opvolgorde van nummer, artiest, datum en
totaal aantal streams. **

In [ ]:
import pandas as pd 
import numpy as np
Dp_spotify= pd.read_csv("SpotifyNL.csv")

print(Dp_spotify.columns)

In [ ]:

Dp_spotify.sort_values(by=["Track.Name", "Artist", "StreamsTotal"],ascending=False)

### 8. 
**StreamsTotal bevat de cumulatieve som van alle views tot en met de betreffende dag. Bereken nu het aantal streams op alleen de aangegeven dag per nummer en artiest.**

*Tip: gebruik diff(). Hoe gaat je om met NA waardes? tip: fillna()*

In [ ]:
Dp_spotify.sort_values(by=['Artist', 'Date'])['StreamsTotal'].diff()


### 9. 
**Bereken de positie in de top 200 o.b.v. aantal streams op die dag
Tip: zet de dataset op volgorde van dag en aantal streams.**

In [ ]:
df = Dp_spotify

df["Date"] = pd.to_datetime(df["Date"], errors="coerce")


df["positie"] = df.groupby("Date")["StreamsTotal"].rank(method="first", ascending=False)

top200 = df[df["positie"] <= 200]

print(top200.head())


### 10. 
**Bepaal de stijging of daling van elk nummer t.o.v. de dag ervoor. Is het nummer nieuw in de
top 200, bereken dan het verschil t.o.v. 201**

In [ ]:

Dp_spotify = Dp_spotify.sort_values(["Date", "Track.Name"])


Dp_spotify["verschil"] = Dp_spotify.groupby("Track.Name")["StreamsTotal"].shift(1)


Dp_spotify["verschil"] = Dp_spotify["verschil"].fillna(201)


print(Dp_spotify.head())
Dp_spotify["verschil"].fillna(201)

Dp_spotify["groep"] = pd.cut(Dp_spotify["positie"], bins=20)
print(Dp_spotify[["Track.Name", "Artist", "StreamsTotal", "positie"]].head(10))




Dp_spotify["Top10"] = Dp_spotify["StreamsTotal"] / 1
avg_df = Dp_spotify.groupby("Date", as_index=False)["Top10"].mean()
top10_days = avg_df.sort_values("Top10", ascending=False).head(10)["Date"]

top10_tracks = Dp_spotify[Dp_spotify["Date"].isin(top10_days)]

print(top10_tracks[["Date", "Track.Name", "Artist", "StreamsTotal"]].sort_values(["Date", "StreamsTotal"], ascending=[True, False]))



In [ ]:

import matplotlib.pyplot as plt

# Maak een histogram van de 'overall' scores in dp_19
plt.hist(dp_19['overall'], bins=20, color='blue', edgecolor='black')

# Voeg labels en een titel toe
plt.title('Histogram van Overall Scores (2019)')
plt.xlabel('Overall Score')
plt.ylabel('Frequentie')

# Toon het histogram
plt.show()